In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
import requests
import hashlib
from binascii import unhexlify, hexlify

Defaulting to user installation because normal site-packages is not writeable


In [18]:
# api-endpoint
URL = "https://blockchain.info/rawblock/000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f?format=hex"
# URL = "https://blockchain.info/rawblock/00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d?format=hex"

# sending get request and saving the response as response object
r = requests.get(url = URL) #, params = PARAMS)
  
# print(r.content)
print('text: ' + r.text)
print('encoding: ' + r.encoding)
# print('type: ' + type(r.text))

blockHex = r.text
# head = r.text[:size2]
curr = 0

def read(size):
    global curr
    prev = curr
    curr += size * 2
#     if size == 1:
#         print("prev: ", prev)
#         print("curr: ", curr)
#         print("head sect: ", blockHex[prev:curr])
    return blockHex[prev:curr]

def readInt(size=4):
    tempArr = bytearray(bytes.fromhex(read(size)))
#     if size == 1:
#         print("tempArr: ", tempArr)
    return int.from_bytes(tempArr, "little")

def readHash(size=32):
    tempArr = bytearray(bytes.fromhex(read(32)))
    tempArr.reverse()
    return tempArr

def readVarInt():
    size = readInt(1)
#     print("readvarint size: ", size)
    if size < 253:
        return size
    elif size == 253:
        return readInt(2)
    elif size == 254:
        return readInt(4)
    elif size == 255:
        return readInt(8)
    return -1

def readTxInput():
    tx_input = {}
    tx_input["prevHash"] = readHash().hex()
    tx_input["txOutId"] = readInt(4)
    tx_input["scriptLen"] = readVarInt()
    tx_input["script"] = read(tx_input["scriptLen"])
    tx_input["seqNo"] = readInt(4)
    return tx_input

def readTxInputs(vin_sz):
    outputs = []
    for i in range(0, vin_sz):
        outputs.append(readTxInput())
    return outputs
    
def readTxOutput():
    tx_output = {}
    tx_output["value"] = readInt(8)
    tx_output["scriptLen"] = readVarInt()
    tx_output["pubkey"] = read(tx_output["scriptLen"])
    return tx_output
    
def readTxOutputs(vout_sz):
    outputs = []
    for i in range(0, vout_sz):
        outputs.append(readTxOutput())
    return outputs
    
def readTx(n_tx):
    outputs = []
    for i in range(0, n_tx):
        tx = {}
        tx["ver"] = readInt()
        tx["vin_sz"] = readVarInt()
        tx["inputs"] = readTxInputs(tx["vin_sz"])
        tx["vout_sz"] = readVarInt()
        tx["outputs"] = readTxOutputs(tx["vout_sz"])
        outputs.append(tx)
    return outputs

print("===HEADER===")

ver = readInt()
print("ver: ", ver)

prev_block = readHash().hex()
print("prev_block: ", prev_block)

mrkl_root = readHash().hex()
print("mrkl_root: ", mrkl_root)

time = readInt()
print("time: ", time)

bits = readInt()
print("bits: ", bits)

nonce = readInt()
print("nonce: ", nonce)

print("===TRANSACTIONS===")

n_tx = readVarInt()
print("n_tx: ", n_tx)

tx = readTx(n_tx)
print("tx: ", tx)

# head2 = unhexlify(head)
# head3 = hashlib.sha256(hashlib.sha256(head2).digest()).digest()
# head4 = hexlify(head3[::-1]).decode("utf-8")
# print('head4: ' + head4)
# toBytes = bytes.fromhex(blockHex)
# print(toBytes)
# toAscii = toBytes.decode("ascii")#("ASCII")

# print(toAscii)

# body = r.text
# body2 = unhexlify(body)
# body3 = hashlib.sha256(hashlib.sha256(body2).digest()).digest()
# body4 = hexlify(body3[::-1]).decode("utf-8")
# print('body4: ' + body4)

# b = bytes.fromhex(blockHex)
# # print(b)
# ba = bytearray(b)
# ba.reverse()
# # print(ba)
# a = ba.decode("ascii")

text: 0100000000000000000000000000000000000000000000000000000000000000000000003ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a29ab5f49ffff001d1dac2b7c0101000000010000000000000000000000000000000000000000000000000000000000000000ffffffff4d04ffff001d0104455468652054696d65732030332f4a616e2f32303039204368616e63656c6c6f72206f6e206272696e6b206f66207365636f6e64206261696c6f757420666f722062616e6b73ffffffff0100f2052a01000000434104678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61deb649f6bc3f4cef38c4f35504e51ec112de5c384df7ba0b8d578a4c702b6bf11d5fac00000000
encoding: utf-8
===HEADER===
ver:  1
prev_block:  0000000000000000000000000000000000000000000000000000000000000000
mrkl_root:  4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b
time:  1231006505
bits:  486604799
nonce:  2083236893
===TRANSACTIONS===
n_tx:  1
tx:  [{'ver': 1, 'vin_sz': 1, 'inputs': [{'prevHash': '0000000000000000000000000000000000000000000000000000000000000000', 'txOutId': 4294967295, 'scr